# **Import Object**

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime

import get_x_y

# **Load Energy**

In [ ]:
energy_6_10_60m = get_x_y.get_targret_energy(1, [6, 7, 8, 9, 10], 3600)
energy_11_1_60m  = get_x_y.get_targret_energy(1, [11, 12, 1], 3600)

energy_6_10_30m = get_x_y.get_targret_energy(1, [6, 7, 8, 9, 10], 1800)
energy_11_1_30m  = get_x_y.get_targret_energy(1, [11, 12, 1], 1800)

# **Load Occupancy**

In [ ]:
occupancy_sum=pd.read_csv('./ecodataset/Occupancy/01_summer.csv')
occupancy_sum=occupancy_sum.set_index('Unnamed: 0')

occupancy_win=pd.read_csv('./ecodataset/Occupancy/01_winter.csv')
occupancy_win=occupancy_win.set_index('Unnamed: 0')

In [ ]:
occupancy_sum = get_x_y.to_intervals_occupancy(occupancy_sum, 3600)
occupancy_win = get_x_y.to_intervals_occupancy(occupancy_win, 3600)

# **Create Ratio**

In [ ]:
ratio_sum = get_x_y.build_ratio(occupancy_sum.columns, energy_6_10_60m , 24)
ratio_win = get_x_y.build_ratio(occupancy_win.columns, energy_11_1_60m , 24)

# **Create Features**

In [6]:
col_sum = pd.DataFrame(ratio_sum.columns.tolist())
energy_sum = get_x_y.get_corresponding_energy(occupancy_sum.columns, energy_6_10_30m)

ratio_sum = ratio_sum.values.T.reshape(-1)

mean_list, max_list, min_list, std_list, range_list = get_x_y.create_features(energy_sum, col_sum)
time_list = [t for t in range(0, 24, 1)] * len(col_sum)
season_list = [1 for i in range(len(ratio_sum))]
weekdays = get_x_y.get_weekdays(occupancy_sum.columns)

features_sum  = pd.DataFrame({
    "Ratio": ratio_sum, "Mean": mean_list, "Max": max_list,
    "Min": min_list, "Std": std_list, "Range": range_list,
    "Time": time_list, "Season": season_list, "Weekday": weekdays
    })

features_sum["Lag-1_Ratio"] = np.concatenate([np.array([0.5]), features_sum["Ratio"].values[:-1]])
features_sum["Lag+1_Ratio"] = np.append(features_sum["Ratio"].values[1:], 0.5)
features_sum = features_sum[(features_sum.Time > 5) & (features_sum.Time < 22)]
features_sum = get_x_y.get_am_pm(features_sum) 
features_sum =  features_sum.reindex([
    'Ratio', 'Mean', 'Max', 'Min',
    'Std', 'Range','Lag-1_Ratio', 'Lag+1_Ratio',
    'Season', 'Weekday', 'Am', 'Lunch', 'Pm'
    ], axis=1)
features_sum.to_csv("1_Sum_X_train.csv", index=False)

occupancy_sum = occupancy_sum[6:22].values.T.reshape(-1) 
occupancy_sum = pd.DataFrame(occupancy_sum, columns=["Occupancy"])
occupancy_sum.to_csv("1_Sum_Y_train.csv", index=False)

In [7]:
col_win = pd.DataFrame(ratio_win.columns.tolist())
energy_win = get_x_y.get_corresponding_energy(occupancy_win.columns, energy_11_1_30m)

ratio_win = ratio_win.values.T.reshape(-1)

mean_list, max_list, min_list, std_list, range_list = get_x_y.create_features(energy_win, col_win)
time_list = [t for t in range(0, 24, 1)] * len(col_win)
season_list = [0 for i in range(len(ratio_win))]
weekdays = get_x_y.get_weekdays(occupancy_win.columns)

features_win = pd.DataFrame({
    "Ratio": ratio_win, "Mean": mean_list, "Max": max_list,
    "Min": min_list, "Std": std_list, "Range": range_list,
    "Time": time_list, "Season": season_list, "Weekday": weekdays
    })

features_win["Lag-1_Ratio"] = np.concatenate([np.array([0.5]), features_win["Ratio"].values[:-1]])
features_win["Lag+1_Ratio"] = np.append(features_win["Ratio"].values[1:], 0.5)
features_win = features_win[(features_win.Time > 5) & (features_win.Time < 22)]
features_win = get_x_y.get_am_pm(features_win)
features_win =  features_win.reindex([
    'Ratio', 'Mean', 'Max', 'Min',
    'Std', 'Range','Lag-1_Ratio', 'Lag+1_Ratio',
    'Season', 'Weekday', 'Am', 'Lunch', 'Pm'
    ], axis=1)
features_win.to_csv("1_Win_X_train.csv", index=False)

occupancy_win = occupancy_win[6:22].values.T.reshape(-1)
occupancy_win = pd.DataFrame(occupancy_win, columns=["Occupancy"])
occupancy_win.to_csv("1_Win_Y_train.csv", index=False)

In [8]:
X = pd.concat([features_sum, features_win], axis=0)
Y = pd.concat([occupancy_sum, occupancy_win], axis=0)
X.to_csv("1_X_train.csv", index=False)
Y.to_csv("1_Y_train.csv", index=False)